# 背景建模

## 帧插法
### 用两帧图像进行差分运算，不同帧对应的像素点相减，判断灰度差的绝对值，当绝对值超过一定阈值是，即可判断为运动目标
#### 问题： 会引入噪音和空洞问题

## 混合高斯模型
##### 类似算法：EM和GMM算法
### 通过学习得到多种背景的高斯分布，并将其融合
### 背景由多个高斯分布融合，每个高斯分布（正态分布）的权重不同
![](pic/1.png)
#### 视频的背景每个像素点应当是符合高斯分布的


### 混合高斯模型学习方法
   1. 首先初始化每个高斯模型矩阵参数
   2. 取视频中T帧数据图像来训练高斯混合模型，来了第一个像素之后用他来当第一个高斯分布
   3. 后面的像素值与前面已有的高斯的均值比较，如果该像素点的值与其模型均值差在3倍方差内，则属于该分部，并对其进行参数更新
   4. 如果下一次来的像素不满足当前的高斯分布，用它来创建新的高斯分布
   （视频中采用3-5个高斯分布基本足够）

### 混合高斯模型测试方法
#### 在测试阶段，对新来像素点的值与混合高斯模型中的每一个均值进行比较，如果其差值在两倍方差之间的话，则认为是背景，否则认为是前景。将前景赋值为255，背景赋值为0.这样就形成一幅前景二值图

In [11]:
import numpy as np
import cv2

cap = cv2.VideoCapture("./video/test.avi")
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2()
_,frame = cap.read()

while _:
    _,frame = cap.read()
    fgmask = fgbg.apply(frame)

    # 开运算去噪点
    fgmask = cv2.morphologyEx(fgmask,cv2.MORPH_OPEN,kernel)

    contours = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[1]

    for c in contours:
        perimeter = cv2.arcLength(c,True) # 闭合矩形
        if perimeter > 150:
            # 筛选掉不必要的轮库
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

    cv2.imshow("frame",frame)
    cv2.imshow("mask",fgmask)
    k = cv2.waitKey(100) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()